## Scraping module
Scraping of straits times news article using selenium that are returned from the search function (keyword - 'food supply')

### Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time

from newspaper import Article
import pandas as pd

### Newspaper Article Summariser

In [2]:
def getSummary(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    return article.summary, article.keywords

### StraitsTimes Scraper

In [10]:
driver = webdriver.Chrome('C:/Users/User/Desktop/FYP/chromedriver.exe')

def getNews():
    output = []
    
    # get news title in a page - need a for loop to get all the pages!
    all_news_per_page=WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"queryly_item_title")))
    
    # get news url (i.e. news source) & news summary & keywords using the summariser module
    for i in range(len(all_news_per_page)):
        news_details = []
        news_details.append(all_news_per_page[i].text)
        xpath = f'//*[@id="resultdata"]/div[{i+2}]/a'        
        url = all_news_per_page[i].find_element_by_xpath(xpath).get_attribute('href')
        news_details.append(url)
        summary = getSummary(url)
        news_details.append(summary[0])
        news_details.append(", ".join(summary[1]))
        output.append(news_details)
    return output

In [11]:
def getStraitsTimesNews(searchTerm):
    base_url = "https://www.straitstimes.com/search?searchkey=" + searchTerm.replace(" ", "%20")
    driver.get(base_url)

    # close ad (if any)
    try:
        iframe=driver.find_elements_by_tag_name("iframe")[3]
        driver.switch_to.frame(iframe)
        driver.find_elements_by_tag_name("iframe")
        ad = driver.switch_to.frame(0)
        driver.find_element_by_id('pclose-btn').click()
        driver.switch_to.default_content()
    except:
        driver.switch_to.default_content()
    
    # filter the search result by date by xpath accordingly
    # within 24 hours, within one week, within 30 days, within a year, within three years
    filter_by_date = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.XPATH,"//*[contains(text(), 'within 30 days')]")))[0].click()

    # get news on page
    output = getNews()
    
    # check for next page
    nextpage = True
    while (nextpage):  
        try:
            nextpage = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,'#resultdata > a > h2')))[0]
            if (nextpage.text == 'Next Page'):
                nextpage.click()
#                 print('clicked next')
                output.extend(getNews())
            else: 
                nextpage = False
        except:
#             print("no next page!")
            nextpage = False
    
    
    # convert to df and export as csv
    df = pd.DataFrame(output, columns=['news title', 'news source(url)', 'content summary', 'keywords'])
    df.to_csv(f"csv_data/{searchTerm}_straitstimes.csv")
    return df

getStraitsTimesNews("food supply")

,news title,news source(url),content summary,keywords
0,"Global food prices hit 2-decade high, threaten...",https://www.straitstimes.com/business/economy/...,WASHINGTON (NYTIMES) - Food prices have skyroc...,"price, weather, world, prices, 2decade, hit, t..."
1,"Ukraine tensions can choke supply of energy, f...",https://www.straitstimes.com/business/companie...,"SINGAPORE - Far away from Singapore, there is ...","spore, singapore, situation, volatility, turn,..."
2,Agri-food sector growing rapidly in S'pore wit...,https://www.straitstimes.com/business/economy/...,SINGAPORE - As the world grapples with challen...,"spore, rapidly, singapore, foods, growing, sus..."
3,Researchers develop blueprint for sustainable ...,https://www.straitstimes.com/singapore/researc...,SINGAPORE - Imagine a future where food waste ...,"singapore, system, researchers, production, bl..."
4,President Xi says country's low-carbon push mu...,https://www.straitstimes.com/asia/east-asia/ch...,SHANGHAI (REUTERS) - China's ambitious low-car...,"emissions, security, push, guarantee, state, x..."
5,Omicron's spread means more food outages at US...,https://www.straitstimes.com/world/united-stat...,WASHINGTON (BLOOMBERG) - The highly contagious...,"seeing, means, workforce, omicron, outages, su..."
6,"Food supply hit ahead of CNY, Ramadan after fl...",https://www.straitstimes.com/asia/se-asia/food...,"""I had to throw away some 1,200 eggs as they c...","malaysian, ahead, hit, farms, month, farm, wip..."
7,Defending the Amazon: How indigenous culture p...,https://www.straitstimes.com/world/defending-t...,"""We've been living here for thousands of years...","protects, rainforest, trees, souza, indigenous..."
8,Colombia demands removal from 'hunger hotspots...,https://www.straitstimes.com/world/colombia-de...,BOGOTA (AFP) - Colombia took umbrage on Monday...,"foreign, removed, migrants, list, included, re..."
9,Trickle of humanitarian aid enters North Korea...,https://www.straitstimes.com/asia/east-asia/tr...,SEOUL (REUTERS) - After nearly two years of bo...,"korea, humanitarian, world, enters, trickle, s..."
